# Case study 2: The Sars-Cov2 model

In [12]:
# from eliater.network_validation import conditional_independence_test_summary
import pandas as pd
from y0.dsl import Variable
from y0.graph import NxMixedGraph
from src.eliater.frontdoor_backdoor import sars_cov2_discrete


This is case study 2 in Figure 6 in this paper: Eliater: an analytical workflow and open source implementation for causal query estimation in biomolecular networks.

In [13]:
graph = NxMixedGraph.from_str_edges(
    directed=[
        ("SARS_COV2", "ACE2"),
        ("ACE2", "Ang"),
        ("Ang", "AGTR1"),
        ("AGTR1", "ADAM17"),
        ("ADAM17", "EGF"),
        ("ADAM17", "TNF"),
        ("ADAM17", "Sil6R"),
        ("SARS_COV2", "PRR"),
        ("PRR", "NFKB"),
        ("EGFR", "NFKB"),
        ("TNF", "NFKB"),
        ("Sil6R", "IL6STAT3"),
        ("Toci", "Sil6R"),
        ("NFKB", "IL6AMP"),
        ("IL6AMP", "cytok"),
        ("IL6STAT3", "IL6AMP"),
        ("EGF", "EGFR"),
        ("Gefi", "EGFR"),
    ],
    undirected=[
        ("SARS_COV2", "Ang"),
        ("ADAM17", "Sil6R"),
        ("PRR", "NFKB"),
        ("EGF", "EGFR"),
        ("EGFR", "TNF"),
        ("EGFR", "IL6STAT3"),
    ],
)

We get the observational data below. However, the data is mixed type, where the exposure (EGFR) is binary and rest of the variables are continuous. Currently, the conditional independence tests in eliater does not support mixed type data. Hence we discretize the data only to use for step 1. Rest of the steps support this type of data, so we use the original data for those steps.

In [14]:
# get observational data
data = sars_cov2_discrete.generate(num_samples=1000, seed=1)

[0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.94784644
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.94784644 0.94784644 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.94784644 0.94784644 0.24973989 0.24973989
 0.24973989 0.24973989 0.94784644 0.94784644 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.94784644 0.24973989 0.24973989
 0.94784644 0.24973989 0.24973989 0.94784644 0.94784644 0.24973989
 0.24973989 0.94784644 0.24973989 0.24973989 0.94784644 0.24973989
 0.94784644 0.24973989 0.24973989 0.24973989 0.94784644 0.94784644
 0.24973989 0.94784644 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.94784644 0.24973989 0.94784644 0.94784644
 0.24973989 0.24973989 0.24973989 0.94784644 0.24973989 0.2497

In [15]:
# # discritize the data
# from sklearn.preprocessing import KBinsDiscretizer
# # discretization transform the raw data
# kbins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
# data_trans = kbins.fit_transform(data)
# print(data_trans)
# column_names = ["SARS_COV2",
#                 "ACE2",
#                 "Ang",
#                 "AGTR1",
#                 "ADAM17",
#                 "Toci",
#                 "Sil6R",
#                 "EGF",
#                 "TNF",
#                 "Gefi",
#                 "EGFR",
#                 "PRR",
#                 "NFKB",
#                 "IL6STAT3",
#                 "IL6AMP",
#                 "cytok",]
# obs_data_discrete = pd.DataFrame(data_trans, columns = [])
# # summarize first few rows


## Step 1: Verify correctness of the network structure

In [16]:
#remove after branches are merged to main
import logging
from typing import Dict, Literal, Optional

import pandas as pd

from y0.algorithm.falsification import get_graph_falsifications
from y0.dsl import Variable
from y0.graph import NxMixedGraph
from y0.struct import get_conditional_independence_tests

logging.basicConfig(format="%(message)s", level=logging.DEBUG)


__all__ = [
    "conditional_independence_test_summary",
    "validate_test",
    "get_state_space_map",
    "is_data_discrete",
    "is_data_continuous",
    "CITest",
    "choose_default_test",
]


def get_state_space_map(
    data: pd.DataFrame, threshold: Optional[int] = 10
) -> Dict[Variable, Literal["discrete", "continuous"]]:
    """Get a dictionary from each variable to its type.

    :param data: the observed data
    :param threshold: The threshold for determining a column as discrete
        based on the number of unique values
    :return: the mapping from column name to its type
    """
    column_values_unique_count = {
        column_name: data[column_name].nunique() for column_name in data.columns
    }
    return {
        Variable(column): "discrete"
        if column_values_unique_count[column] <= threshold
        else "continuous"
        for column in data.columns
    }


def is_data_discrete(data: pd.DataFrame) -> bool:
    """Check if all the columns in the dataframe has discrete data.

    :param data: observational data.
    :return: True, if all the columns have discrete data, False, otherwise
    """
    variable_types = set(get_state_space_map(data=data).values())
    return variable_types == {"discrete"}


def is_data_continuous(data: pd.DataFrame) -> bool:
    """Check if all the columns in the dataframe has continuous data.

    :param data: observational.
    :return: True, if all the columns have continuous data, False, otherwise
    """
    variable_types = set(get_state_space_map(data).values())
    return variable_types == {"continuous"}


CITest = Literal[
    "pearson",
    "chi-square",
    "cressie_read",
    "freeman_tuckey",
    "g_sq",
    "log_likelihood",
    "modified_log_likelihood",
    "power_divergence",
    "neyman",
]


def choose_default_test(data: pd.DataFrame) -> CITest:
    """Choose the default statistical test for testing conditional independencies based on the data.

    :param data: observational data.
    :return: the default test based on data
    :raises NotImplementedError: if data is of mixed type (contains both discrete and continuous columns)
    """
    if is_data_discrete(data):
        return "chi-square"
    if is_data_continuous(data):
        return "pearson"
    raise NotImplementedError(
        "Mixed data types are not allowed. Either all of the columns of data should be discrete / continuous."
    )


def validate_test(
    data: pd.DataFrame,
    test: Optional[CITest],
) -> None:
    """Validate the conditional independency test passed by the user.

    :param data: observational data.
    :param test: the conditional independency test passed by the user.
    :raises ValueError: if the passed test is invalid / unsupported, pearson is used for discrete data or
        chi-square is used for continuous data
    """
    tests = get_conditional_independence_tests()
    if test not in tests:
        raise ValueError(f"`{test}` is invalid. Supported CI tests are: {sorted(tests)}")

    if is_data_continuous(data) and test != "pearson":
        raise ValueError(
            "The data is continuous. Either discretize and use chi-square or use the pearson."
        )

    if is_data_discrete(data) and test == "pearson":
        raise ValueError("Cannot run pearson on discrete data. Use chi-square instead.")


def conditional_independence_test_summary(
    graph: NxMixedGraph,
    data: pd.DataFrame,
    test: Optional[CITest] = None,
    significance_level: Optional[float] = None,
    verbose: Optional[bool] = False,
) -> None:
    """Print the summary of conditional independency test results.

    Prints the summary to the console, which includes the total number of conditional independence tests,
    the number and percentage of failed tests, and statistical information about each test such as p-values,
    and test results.

    :param graph: an NxMixedGraph
    :param data: observational data corresponding to the graph
    :param test: the conditional independency test to use. If None, defaults to ``pearson`` for continuous data
        and ``chi-square`` for discrete data.
    :param significance_level: The statistical tests employ this value for
        comparison with the p-value of the test to determine the independence of
        the tested variables. If none, defaults to 0.01.
    :param verbose: If `False`, only print the details of failed tests.
        If 'True', print the details of all the conditional independency results. Defaults to `False`
    :raises NotImplementedError: if data is of mixed type (contains both discrete and continuous columns)
    """
    if significance_level is None:
        significance_level = 0.01
    if not test:
        test = choose_default_test(data)
    else:
        # Validate test and data
        validate_test(data=data, test=test)
        if len(set(get_state_space_map(data).values())) > 1:
            raise NotImplementedError(
                "Mixed data types are not allowed. Either all of the columns of data should be discrete / continuous."
            )
    test_results = get_graph_falsifications(
        graph=graph, df=data, method=test, significance_level=significance_level
    ).evidence
    # Find the result based on p-value
    test_results["result"] = test_results["p"].apply(
        lambda p_value: "fail" if p_value < significance_level else "pass"
    )
    # Selecting columns of interest
    test_results = test_results[["left", "right", "given", "p", "result"]]
    # Sorting the rows by index
    test_results = test_results.sort_index()
    test_results = test_results.rename(columns={"p": "p-value"})
    failed_tests = test_results[test_results["result"] == "fail"]
    total_no_of_tests = len(test_results)
    total_no_of_failed_tests = len(failed_tests)
    percentage_of_failed_tests = total_no_of_failed_tests / total_no_of_tests * 100
    logging.info("Total number of conditional independencies: " + str(total_no_of_tests))
    logging.info("Total number of failed tests: " + str(total_no_of_failed_tests))
    logging.info("Percentage of failed tests: " + str(percentage_of_failed_tests))
    if verbose:
        logging.info(test_results.to_string(index=False))
    else:
        logging.info(failed_tests.to_string(index=False))

In [7]:
conditional_independence_test_summary(graph=graph, data=data, verbose=False)

KeyboardInterrupt: 

In [8]:
test_results = get_graph_falsifications(
        graph=graph, df=data, method="chi-square", significance_level=0.01
    ).evidence

KeyboardInterrupt: 

## Step 2: Check query identifiability

In [17]:
from y0.algorithm.identify import Identification
from y0.dsl import P
id_in = Identification.from_expression(
    query=P(Variable('cytok') @ Variable('EGFR')),
    graph=graph,
)
id_in

Identification(outcomes="{cytok}, treatments="{EGFR}",conditions="set()",  graph="NxMixedGraph(directed=<networkx.classes.digraph.DiGraph object at 0x1116d3010>, undirected=<networkx.classes.graph.Graph object at 0x1116d1290>)", estimand="P(ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6R, TNF, Toci, cytok)")

The query is identifiable.

## Step 3: Find nuisance variables and mark them as latent

In [18]:
#remove after merging simplify branch to main
import itertools
from typing import Iterable, Optional, Set, Union

import networkx as nx

from y0.algorithm.simplify_latent import simplify_latent_dag
from y0.dsl import Variable
from y0.graph import DEFAULT_TAG, NxMixedGraph

__all__ = [
    "remove_latent_variables",
    "mark_nuisance_variables_as_latent",
    "find_all_nodes_in_causal_paths",
    "find_nuisance_variables",
]


def remove_latent_variables(
    graph: NxMixedGraph,
    treatments: Union[Variable, Set[Variable]],
    outcomes: Union[Variable, Set[Variable]],
    tag: Optional[str] = None,
) -> NxMixedGraph:
    """Find all nuissance variables and remove them based on Evans' simplification rules.

    :param graph: an NxMixedGraph
    :param treatments: a list of treatments
    :param outcomes: a list of outcomes
    :param tag: The tag for which variables are latent
    :return: the new graph after simplification
    """
    lv_dag = mark_nuisance_variables_as_latent(
        graph=graph, treatments=treatments, outcomes=outcomes, tag=tag
    )
    simplified_latent_dag = simplify_latent_dag(lv_dag, tag=tag)
    return NxMixedGraph.from_latent_variable_dag(simplified_latent_dag.graph, tag=tag)


def mark_nuisance_variables_as_latent(
    graph: NxMixedGraph,
    treatments: Union[Variable, Set[Variable]],
    outcomes: Union[Variable, Set[Variable]],
    tag: Optional[str] = None,
) -> nx.DiGraph:
    """Find all the nuisance variables and mark them as latent.

    Mark nuisance variables as latent by first identifying them, then creating a new graph where these
    nodes are marked as latent. Nuisance variables are the descendants of nodes in all proper causal paths
    that are not ancestors of the outcome variables nodes. A proper causal path is a directed path from
    treatments to the outcome. Nuisance variables should not be included in the estimation of the causal
    effect as they increase the variance.

    :param graph: an NxMixedGraph
    :param treatments: a list of treatments
    :param outcomes: a list of outcomes
    :param tag: The tag for which variables are latent
    :return: the modified graph after simplification, in place
    """
    if tag is None:
        tag = DEFAULT_TAG
    nuisance_variables = find_nuisance_variables(graph, treatments=treatments, outcomes=outcomes)
    lv_dag = NxMixedGraph.to_latent_variable_dag(graph, tag=tag)
    # Set nuisance variables as latent
    for node, data in lv_dag.nodes(data=True):
        if Variable(node) in nuisance_variables:
            data[tag] = True
    return lv_dag


def find_all_nodes_in_causal_paths(
    graph: NxMixedGraph,
    treatments: Union[Variable, Set[Variable]],
    outcomes: Union[Variable, Set[Variable]],
) -> Set[Variable]:
    """Find all the nodes in proper causal paths from treatments to outcomes.

    A proper causal path is a directed path from treatments to the outcome.

    :param graph: an NxMixedGraph
    :param treatments: a list of treatments
    :param outcomes: a list of outcomes
    :return: the nodes on all causal paths from treatments to outcomes.
    """
    if isinstance(treatments, Variable):
        treatments = {treatments}
    if isinstance(outcomes, Variable):
        outcomes = {outcomes}

    return {
        node
        for treatment, outcome in itertools.product(treatments, outcomes)
        for causal_path in nx.all_simple_paths(graph.directed, treatment, outcome)
        for node in causal_path
    }


def find_nuisance_variables(
    graph: NxMixedGraph,
    treatments: Union[Variable, Set[Variable]],
    outcomes: Union[Variable, Set[Variable]],
) -> Iterable[Variable]:
    """Find the nuisance variables in the graph.

    Nuisance variables are the descendants of nodes in all proper causal paths that are
    not ancestors of the outcome variables' nodes. A proper causal path is a directed path
    from treatments to the outcome. Nuisance variables should not be included in the estimation
    of the causal effect as they increase the variance.

    :param graph: an NxMixedGraph
    :param treatments: a list of treatments
    :param outcomes: a list of outcomes
    :returns: The nuisance variables.
    """
    if isinstance(treatments, Variable):
        treatments = {treatments}
    if isinstance(outcomes, Variable):
        outcomes = {outcomes}

    # Find the nodes on all causal paths
    nodes_on_causal_paths = find_all_nodes_in_causal_paths(
        graph=graph, treatments=treatments, outcomes=outcomes
    )

    # Find the descendants of interest
    descendants_of_nodes_on_causal_paths = graph.descendants_inclusive(nodes_on_causal_paths)

    # Find the ancestors of outcome variables
    ancestors_of_outcomes = graph.ancestors_inclusive(outcomes)

    descendants_not_ancestors = descendants_of_nodes_on_causal_paths.difference(
        ancestors_of_outcomes
    )

    nuisance_variables = descendants_not_ancestors.difference(treatments.union(outcomes))
    return nuisance_variables

This function finds the nuisance variables for the input graph.

In [19]:
nuisance_variables = find_nuisance_variables(graph, treatments=Variable("EGFR"), outcomes=Variable("cytok"))
nuisance_variables

set()

No variable is identified as the nuisance variable. Hence the simplified network in the next step will produce a graph similar to the original graph.

## Step 4: Simplify the network

The following function find the nuisance variable (step 3), marks them as latent and then applies Evan's simplification rules to remove the nuisance variables. As there are no nuisance variables, the new graph will be the same as the original graph.

In [20]:
new_graph = remove_latent_variables(graph, treatments=Variable("EGFR"), outcomes=Variable("cytok"))

## Step 5: Estimate the query

In [21]:
#remove after estimation branch is merged to main
#pip install ananke-causal

In [22]:
#remove after estimation branch is merged to main and replace with eliater codes
from ananke.graphs import ADMG
from ananke.identification import OneLineID
from ananke.estimation import CausalEffect
from ananke.datasets import load_afixable_data
from ananke.estimation import AutomatedIF
from ananke import identification
import numpy as np
import pandas as pd
from ananke.models import LinearGaussianSEM
import warnings
import numpy
import random
warnings.filterwarnings('ignore')

In [23]:
#remove after estimation branch is merged to main
vertices = ['SARS_COV2', 'ACE2','Ang','AGTR1',
            'ADAM17','Toci','Sil6R','EGF',
            'TNF','EGFR','PRR','NFKB',
            'IL6STAT3','IL6AMP','cytok','Gefi']
di_edges = [('SARS_COV2','ACE2'),
            ('ACE2','Ang'),
            ('Ang','AGTR1'),
            ('AGTR1','ADAM17'),
            ('ADAM17','EGF'),
            ('ADAM17','TNF'),
            ('ADAM17','Sil6R'),
            ('SARS_COV2','PRR'),
            ('PRR','NFKB'),
            ('EGFR','NFKB'),
            ('TNF','NFKB'),
            ('Sil6R','IL6STAT3'),
            ('Toci','Sil6R'),
            ('NFKB','IL6AMP'),
            ('IL6AMP','cytok'),
            ('IL6STAT3','IL6AMP'),
            ('EGF','EGFR'), 
            ('Gefi', 'EGFR')]
bi_edges = [("SARS_COV2", "Ang"),
            ("ADAM17", "Sil6R"),
            ("PRR", "NFKB"),
            ("EGF", "EGFR"),
            ("EGFR", "TNF"),
            ("EGFR", "IL6STAT3")]
new_graph = ADMG(vertices, di_edges, bi_edges)

ate_obj = CausalEffect(graph=new_graph, treatment='EGFR', outcome='cytok')  # setting up the CausalEffect object
ate_obj

SG
ADMG
SG
ADMG
SG
ADMG
SG
ADMG
SG
ADMG
SG
ADMG



 Treatment is a-fixable.

 Available estimators are :
 
1. IPW (ipw)
2. Outcome regression (gformula)
3. Generalized AIPW (aipw)
 
Suggested estimator is Generalized AIPW 


In [25]:
#remove after estimation branch is merged to main and use eliater function
#Estimated ATE
ate_obj.compute_effect(data, "aipw")

SG
ADMG
SG
ADMG


-0.21594360092118656

In [27]:
#get the real value of ATE
intv_data1 = sars_cov2_discrete.generate(num_samples=1000, treatments={Variable('EGFR'):1}, seed=1)
intv_data0 = sars_cov2_discrete.generate(num_samples=1000, treatments={Variable('EGFR'):0}, seed=1)
np.mean(intv_data1['cytok']) - np.mean(intv_data0['cytok'])

[0.94784644 0.24973989 0.24973989 0.94784644 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.94784644 0.24973989 0.24973989 0.24973989 0.94784644 0.24973989
 0.24973989 0.94784644 0.94784644 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.94784644 0.24973989
 0.94784644 0.24973989 0.24973989 0.24973989 0.94784644 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989 0.94784644
 0.24973989 0.24973989 0.24973989 0.94784644 0.24973989 0.24973989
 0.94784644 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.24973989 0.24973989 0.24973989 0.94784644 0.24973989
 0.94784644 0.24973989 0.24973989 0.24973989 0.24973989 0.24973989
 0.24973989 0.94784644 0.24973989 0.24973989 0.94784644 0.2497

-0.018000000000000016